In [1]:
import numpy as np
import pandas as pd

In [2]:
#importing PyTorch 
import torch 
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [4]:
training_set = training_set.values
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]], dtype=int64)

In [5]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set.head()

,1,6,5,887431973
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883


In [6]:
test_set = test_set.values
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]], dtype=int64)

In [7]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_users

943

In [8]:
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))
nb_movies

1682

In [9]:
#creating the data structure 
def convert(data):
    res = []
    for i in range(nb_users):
        movies = np.zeros(nb_movies)
        for j in range(len(data)):
            if data[j, 0] == i+1:
                movies[data[j, 1] - 1] = data[j, 2]
        res.append(movies)
    return np.array(res)

In [10]:
training_set = convert(training_set)
test_set = convert(test_set)

In [11]:
training_set = torch.FloatTensor(training_set)
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [12]:
test_set = torch.FloatTensor(test_set)
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [13]:
#creating the architecture of our Autoencoders 
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [14]:
sae = SAE()
criterion = nn.MSELoss() 
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [15]:
type(sae)

__main__.SAE

In [16]:
type(criterion)

torch.nn.modules.loss.MSELoss

In [18]:
#training our SAE
epochs = 200
for epoch in range(1, epochs + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input_vector = Variable(training_set[id_user]).unsqueeze(0)
        target_vector = input_vector.clone()
        if torch.sum(target_vector.data > 0) > 0:
            output_vector = sae.forward(input_vector)
            target_vector.require_grad = False
            output_vector[target_vector == 0] = 0
            loss = criterion(output_vector, target_vector)
            mean_corrector = nb_movies/float(torch.sum(target_vector.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data * mean_corrector)
            s += 1.
            optimizer.step()
    print("epoch: " + str(epoch) + " loss: " + str(train_loss/s))

epoch: 1 loss: tensor(1.7709)
epoch: 2 loss: tensor(1.0967)
epoch: 3 loss: tensor(1.0533)
epoch: 4 loss: tensor(1.0383)
epoch: 5 loss: tensor(1.0308)
epoch: 6 loss: tensor(1.0265)
epoch: 7 loss: tensor(1.0238)
epoch: 8 loss: tensor(1.0220)
epoch: 9 loss: tensor(1.0205)
epoch: 10 loss: tensor(1.0196)
epoch: 11 loss: tensor(1.0188)
epoch: 12 loss: tensor(1.0185)
epoch: 13 loss: tensor(1.0176)
epoch: 14 loss: tensor(1.0176)
epoch: 15 loss: tensor(1.0171)
epoch: 16 loss: tensor(1.0171)
epoch: 17 loss: tensor(1.0168)
epoch: 18 loss: tensor(1.0164)
epoch: 19 loss: tensor(1.0165)
epoch: 20 loss: tensor(1.0161)
epoch: 21 loss: tensor(1.0159)
epoch: 22 loss: tensor(1.0158)
epoch: 23 loss: tensor(1.0158)
epoch: 24 loss: tensor(1.0157)
epoch: 25 loss: tensor(1.0158)
epoch: 26 loss: tensor(1.0156)
epoch: 27 loss: tensor(1.0153)
epoch: 28 loss: tensor(1.0151)
epoch: 29 loss: tensor(1.0127)
epoch: 30 loss: tensor(1.0114)
epoch: 31 loss: tensor(1.0092)
epoch: 32 loss: tensor(1.0097)
epoch: 33 loss: t

In [19]:
np.shape(test_set)

torch.Size([943, 1682])

In [21]:
#Testing our model 
test_loss = 0
counter = 0.
for id_user in range(nb_users):
    input_vector = Variable(training_set[id_user]).unsqueeze(0)
    target_vector = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target_vector.data > 0) > 0:
        output_vector = sae.forward(input_vector)
        output_vector[target_vector == 0] = 0
        loss = criterion(output_vector, target_vector)
        mean_corrector = nb_movies/float(torch.sum(target_vector.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data * mean_corrector)
        counter += 1.
print("loss : " + str(test_loss/counter))

loss : tensor(0.9531)


In [22]:
output_vector[0]

tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SelectBackward>)

In [23]:
target_vector[0]

tensor([0., 0., 0.,  ..., 0., 0., 0.])